In [96]:
import pandas as pd
import praw
from praw.models import MoreComments
import re
from nltk.tokenize import word_tokenize, RegexpTokenizer


In [97]:
CLIENT_ID = "ZOp9gH6lJ26wOg"
CLIENT_SECRET = "RvKRAGfps4AuP9b7t4jEmh_v09cB4A" 

reddit = praw.Reddit(
  client_id = CLIENT_ID,
  client_secret = CLIENT_SECRET,
  user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
)

In [98]:
sub = "wallstreetbets"
posts = reddit.subreddit(sub).hot(limit = 500)
data = []
for p in posts:
    data.append([p.title, p.selftext, p.score, p.num_comments, p.url])
    
data = pd.DataFrame(data = data, columns=["title", "text", "upvotes", "num_comments", "url"])
data.head()

,title,text,upvotes,num_comments,url
0,"What Are Your Moves Tomorrow, April 09, 2021",Your daily trading discussion thread. Please k...,446,12536,https://www.reddit.com/r/wallstreetbets/commen...
1,WSB Rules - Please Read Before Posting,# Welcome to WSB!\n\n\---\n\nAdditional readin...,202,65,https://www.reddit.com/r/wallstreetbets/commen...
2,"I told my wife I’ve only invested $5,000. Happ...",,12187,1059,https://i.redd.it/ajm5pemrt1s61.jpg
3,Robinhood failed to disclose certain trading d...,,2949,153,https://markets.businessinsider.com/news/stock...
4,CHARLES KNOWS WHATS GOOD... WE DIDNT RUN AND H...,,46044,1506,https://v.redd.it/kdaw80nc30s61


In [100]:
def get_comments(url):
    submission = reddit.submission(url=url)
    comments = []
    # Iterate through posts and get comments. NOt all comments have ".body" because thety could be "load more"
    for top_level_comment in submission.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        comments.append(top_level_comment.body)
    return comments
        
test_url = data.iloc[0, 4]
#print(get_comments(test_url)[:5])

In [105]:
df = pd.read_csv("./all_stocks.csv")
ticker_set = set(df["Symbol"].values)
ticker_counts = {}
ticker_posts = []

# TODO: fine tune words to ignore
ignore_words = set(["ON", "IT", "AT", "ARE", "FOR", "YOLO", "PUMP", "RH", "EOD", "IPO", "ATH", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", 
        "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"])

for index, row in data.iterrows():
    title = row["title"]# + " " + row["text"]
    words = re.findall(r'[A-Z]+', title)
    #words = re.findall(r'[A-Za-x][\S]*', title)
    ticks = [w.upper() for w in words if w.upper() in ticker_set - ignore_words]
    for t in ticks:
        if t in ticker_counts:
            ticker_counts[t] += 1
        else:
            ticker_counts[t] = 1 
        ticker_posts.append([t, title])
    

ticker_posts = pd.DataFrame(data = ticker_posts, columns = ["ticker", "text"])
    

In [106]:
top = sorted(ticker_counts.items(), key = lambda x: x[1], reverse = True)
top[:10]

[('GME', 28),
 ('PLTR', 9),
 ('ASO', 8),
 ('DD', 8),
 ('VIAC', 7),
 ('RKT', 5),
 ('BB', 4),
 ('FUBO', 3),
 ('AMD', 3),
 ('AAPL', 3)]

In [109]:
ticker_posts.head()

,ticker,text
0,GOOD,CHARLES KNOWS WHATS GOOD... WE DIDNT RUN AND H...
1,RUN,CHARLES KNOWS WHATS GOOD... WE DIDNT RUN AND H...
2,GME,GME Hype for the retards
3,GME,How to vote in GME shareholders meeting For Fi...
4,GME,"GME Megathread for April 08, 2021"
